In [1]:
from fastai.vision.all import *
import pandas as pd
import cam
import util

model_dir = Path('../saves/dam')

In [2]:
dls, labels = util.chexpert_data_loader(bs=16)

FileNotFoundError: [Errno 2] No such file or directory: '/storage/archive/CheXpert-v1.0-small.zip'

# Base model

In [ ]:
chexpert_learner_base = util.ChexpertLearner(dls, densenet121, n_out=len(labels),
                                        loss_func=BCEWithLogitsLossFlat(),
                                        model_dir=Path('auc_maximization'),
                                        metrics=[RocAucMulti(average=None),
                                                 RocAucMulti(average='weighted')])

In [ ]:
chexpert_learner_base.find_lr()

In [ ]:
chexpert_learner_base.learn_model(use_saved=False, epochs=1)

In [ ]:
# Save the body of the trained base model
torch.save(chexpert_learner_base.learn.model[0].state_dict(), model_dir/'chexpert_learner_base.pth')

# DAM model

In [ ]:
# Create a new densenet121 model
model_dam = cnn_learner(dls, densenet121, n_out=len(labels), y_range=[0,1]).model

# Replace the body of new model with the body from the base model
model_dam[0].load_state_dict(torch.load(model_dir/'chexpert_learner_base.pth'))

# Set all parameters to require grad
for param in model_dam.parameters():
    param.requires_grad = True

# Move model to cuda
model_dam = model_dam.cuda()

In [ ]:
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG

gamma = 500
weight_decay = 0
margin = 1.0

loss_func = AUCMLoss()
opt_func = PESG(model_dam, a=loss_func.a, b=loss_func.b, alpha=loss_func.alpha, gamma=gamma, margin=margin)

In [ ]:
import time
import pickle
from sklearn.metrics import roc_auc_score

def train(model, dls, loss_func, opt_func, max_epoch=2, checkpoint=None):
    # Load checkpoint if available
    if checkpoint != None:
        model_checkpoint = torch.load(checkpoint)
        model.load_state_dict(model_checkpoint['state_dict'])
        opt_func.load_state_dict(model_checkpoint['optimizer'])
    
    # Train model
    max_auc = 0
    
    for epoch in range(max_epoch):

        train_pred = []
        train_true = []
        model.train()    
        for i, (data, targets) in enumerate(dls.train):
            opt_func.zero_grad()
            y_pred = model(data)
            loss = loss_func(y_pred, targets)
            loss.backward(retain_graph=True)
            opt_func.step()

            train_pred.append(y_pred.cpu().detach().numpy())
            train_true.append(targets.cpu().detach().numpy())
            
            if i == 2000 or i == 8000:
                opt_func.lr = opt_func.lr/3
                opt_func.update_regularizer()

        train_true = np.concatenate(train_true)
        train_pred = np.concatenate(train_pred)
        pickle.dump({'train_true': train_true, 'train_pred': train_pred}, open(model_dir/f'train_metrics_{epoch}.p', 'wb'))
        train_auc = roc_auc_score(train_true, train_pred) 
        
        # Eval model
        model.eval()
        test_pred = []
        test_true = [] 
        for j, (test_data, test_targets) in enumerate(dls.valid):
            y_pred = model(test_data)
            test_pred.append(y_pred.cpu().detach().numpy())
            test_true.append(test_targets.numpy())
        test_true = np.concatenate(test_true)
        test_pred = np.concatenate(test_pred)
        val_auc =  roc_auc_score(test_true, test_pred) 

        # print results
        print("epoch: {}, train_loss: {:4f}, train_auc:{:4f}, test_auc:{:4f}, lr:{:4f}".format(epoch, loss.item(), train_auc, val_auc, optimizer.lr ))
        
        # Save checkpoint
        if val_auc > max_auc:
            max_auc = val_auc
            torch.save({
                'epoch': epoch+1,
                'best_auc': val_auc,
                'state_dict': model.state_dict(),
                'optimizer' : opt_func.state_dict()
            }, f"m-epoch {epoch+1}-{time.strftime('%Y_%h_%d-%H_%M_%S')}.pth.tar")

In [ ]:
train(model_dam, dls, loss_func, opt_func)